#### Datasource: https://stooq.com/db/h/

In [6]:
import os
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import mplfinance as mpf
from datetime import datetime

%matplotlib inline

In [12]:
PATH = r'./data/daily/'
COUNTRY = r'de'
MARKET = r'xetra stocks'
TICKER = 'bayn'
FILE_CSV = PATH+COUNTRY+"/"+MARKET+"/1/"+TICKER+".csv"

# APPLE_FILE_CSV = r'../data/stooq/data/daily/us/nasdaq stocks/1/aapl.csv'
# APPLE_FILE_CSV = r'../data/stooq/data/daily/de/xetra stocks/1/bayn.csv'
print(FILE_CSV)
# APPLE_FILE = r'./data/daily/us/nasdaq stocks/1/aapl.us.txt'
# APPLE_FILE = r'./data/daily/de/xetra stocks/1/bayn.de.txt'
FILE = r'./data/daily/us/nyse etfs/voo.us.txt'
# APPLE_FILE_CSV = r'./data/daily/us/nasdaq stocks/1/aapl.csv'
# APPLE_FILE_CSV = r'./data/daily/de/xetra stocks/1/bayn.csv'
FILE_CSV = r'./data/daily/us/nyse etfs/voo.csv'
dateparse = lambda x: datetime.strptime(x, '%Y%m%d')

./data/daily/de/xetra stocks/1/bayn.csv


In [47]:
def convert_to_csv(path, country, folder, *tickers):
    dateparse = lambda x: datetime.strptime(x, '%Y%m%d')
    
    for ticker in tickers:
        print("tikcer: "+ticker)
        filename = ticker+"."+country+".txt"
        filename_csv = ticker+".csv"
        full_path = os.path.join(path+country,folder)
    
        for root, dirs, files in os.walk(full_path):
            found_path = os.path.join(root, filename)
            if os.path.isfile(found_path):
                print("File exists: "+found_path)
                file = pd.read_csv(os.path.join(root, filename), usecols=["<DATE>","<OPEN>","<HIGH>","<LOW>","<CLOSE>","<VOL>","<OPENINT>"],parse_dates=['<DATE>'], date_parser=dateparse)
                file.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'OpenInt']
                full_filename_csv = os.path.join(root, filename_csv)
                file.to_csv(full_filename_csv, index = False, date_format='%Y-%m-%d %H:%M:%S')
                yield file

# ticker_dfs = [df for df in convert_to_csv(PATH, "de", 'xetra stocks', 'bayn')]
ticker_dfs = [df for df in convert_to_csv(PATH, "de", 'xetra stocks', 'bayn', 'bas')]
ticker_dfs

tikcer: bayn
File exists: ./data/daily/de/xetra stocks/1/bayn.de.txt
tikcer: bas
File exists: ./data/daily/de/xetra stocks/1/bas.de.txt


[           Date       Open       High        Low      Close   Volume  OpenInt
 0    2009-09-23  31.323012  31.626543  30.867717  31.092065  2744656        0
 1    2009-09-24  30.907308  31.349406  30.122088  30.471808  4208530        0
 2    2009-09-25  30.682959  30.682959  29.917535  30.498202  3035204        0
 3    2009-09-28  30.287050  31.969664  30.023111  31.969664  5466884        0
 4    2009-09-29  31.936672  32.490944  31.428588  32.227005  4677100        0
 ...         ...        ...        ...        ...        ...      ...      ...
 2917 2021-03-30  53.510000  53.810000  53.070000  53.750000  2370698        0
 2918 2021-03-31  53.760000  54.210000  53.500000  53.960000  2977151        0
 2919 2021-04-01  53.970000  54.090000  53.220000  53.230000  3641900        0
 2920 2021-04-06  53.400000  54.260000  53.280000  53.780000  3632619        0
 2921 2021-04-07  53.780000  54.100000  53.510000  53.510000  2561319        0
 
 [2922 rows x 7 columns],
            Date       O

In [8]:
apple = pd.read_csv(APPLE_FILE, usecols=["<DATE>","<OPEN>","<HIGH>","<LOW>","<CLOSE>","<VOL>","<OPENINT>"],parse_dates=['<DATE>'], date_parser=dateparse)
apple.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'OpenInt']
apple.head()
apple.to_csv(APPLE_FILE_CSV, index = False, date_format='%Y-%m-%d %H:%M:%S')

In [9]:
apple_csv = pd.read_csv(APPLE_FILE_CSV)
apple_csv.head()

,Date,Open,High,Low,Close,Volume,OpenInt
0,2010-09-14 00:00:00,84.386,84.910,84.007,84.549,72412,0
1,2010-09-15 00:00:00,84.203,84.827,84.022,84.763,11315,0
2,2010-09-16 00:00:00,84.533,84.781,84.269,84.729,72610,0
3,2010-09-17 00:00:00,85.237,85.237,84.533,84.812,60465,0
4,2010-09-20 00:00:00,85.125,86.191,84.894,86.191,23164,0


In [ ]:
apple = pd.read_csv(APPLE_FILE,index_col='<DATE>',parse_dates=['<DATE>'], date_parser=dateparse)
apple.columns = ['Ticker', 'PER', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'OpenInt']
apple.head()

In [ ]:
apple['EMA12'] = apple['Close'].ewm(span=12).mean()
apple['EMA26'] = apple['Close'].ewm(span=26).mean()
apple['MACD'] = apple['EMA26'] - apple['EMA12']

In [ ]:
apple['MACDSignalLine'] = apple['MACD'].ewm(span=9).mean()
apple['Histogram'] = apple['MACD'] - apple['MACDSignalLine']

In [ ]:
apds = [mpf.make_addplot(apple['EMA12'][-100:], color='yellow'),
        mpf.make_addplot(apple['EMA26'][-100:], color='c'),
        mpf.make_addplot(apple['MACD'][-100:], panel=1, secondary_y=True, color='orange'),
        mpf.make_addplot(apple['MACDSignalLine'][-100:], panel=1, secondary_y=True, color='blue'),
        mpf.make_addplot(apple['Histogram'][-100:], panel=1, type='bar', color='purple', secondary_y=False)]


In [ ]:
mc = mpf.make_marketcolors(up='#00ff00',down='#ff2e2e',inherit=True)
s  = mpf.make_mpf_style(base_mpf_style='nightclouds',marketcolors=mc)

fig, axes = mpf.plot(apple[-100:], 
                     volume_panel = 2,
                     figratio=(1.5,1),
                     figscale=1, 
                     type='candle', 
                     style=s,
                     addplot=apds,
                     returnfig=True)

labels = ['EMA12', 'EMA26']

axes[0].legend(labels, loc='upper left')
axes[2].legend(['MACD - Signal'], fontsize=6, loc='upper right')
axes[3].legend(['MACD', 'Signal Line'], fontsize=6, loc='lower left')

axes[0].set_title('Apple Chart')

In [ ]:
def RSI(series, period):
    delta = series.diff().dropna()
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    u[u.index[period-1]] = np.mean( u[:period] ) 
    u = u.drop(u.index[:(period-1)])
    d[d.index[period-1]] = np.mean( d[:period] ) 
    d = d.drop(d.index[:(period-1)])
    rs = pd.DataFrame.ewm(u, com=period-1, adjust=False).mean() / \
         pd.DataFrame.ewm(d, com=period-1, adjust=False).mean()
    return 100 - 100 / (1 + rs)

In [ ]:
apple['RSI'] = RSI(apple['Close'], 14)

In [ ]:
apple.tail(10)

In [ ]:
apds2 = [mpf.make_addplot(apple['EMA12'][-100:], color='yellow'),
        mpf.make_addplot(apple['EMA26'][-100:], color='c'),
        mpf.make_addplot(apple['RSI'][-100:], panel=2, secondary_y=True, color='green'),
        mpf.make_addplot(apple['MACD'][-100:], panel=1, secondary_y=True, color='orange'),
        mpf.make_addplot(apple['MACDSignalLine'][-100:], panel=1, secondary_y=True, color='blue'),
        mpf.make_addplot(apple['Histogram'][-100:], panel=1, type='bar', color='purple', secondary_y=False)]


In [ ]:
mc = mpf.make_marketcolors(up='#00ff00',down='#ff2e2e',inherit=True)
s  = mpf.make_mpf_style(base_mpf_style='nightclouds',marketcolors=mc)

fig, axes = mpf.plot(apple[-100:], 
                     volume_panel = 3,
                     figratio=(1.5,1),
                     figscale=1, 
                     type='candle', 
                     style=s,
                     addplot=apds2,
                     returnfig=True)

labels = ['EMA12', 'EMA26']

axes[0].legend(labels, loc='upper left')
axes[3].legend(['RSI'], fontsize=6, loc='lower right')
axes[2].legend(['MACD'], fontsize=6, loc='upper right')

axes[0].set_title('Apple Chart')